函数是Python内建支持的一种封装，我们通过把大段代码拆成函数，通过一层一层的函数调用，就可以把复杂任务分解成简单的任务，这种分解可以称之为面向过程的程序设计。函数就是面向过程的程序设计的基本单元。

而函数式编程（请注意多了一个“式”字）——Functional Programming，虽然也可以归结到面向过程的程序设计，但其思想更接近数学计算。

函数式编程就是一种抽象程度很高的编程范式，纯粹的函数式编程语言编写的函数没有变量，因此，任意一个函数，只要输入是确定的，输出就是确定的，这种纯函数我们称之为没有副作用。而允许使用变量的程序设计语言，由于函数内部的变量状态不确定，同样的输入，可能得到不同的输出，因此，这种函数是有副作用的。

Python对函数式编程提供部分支持。由于Python允许使用变量，因此，Python不是纯函数式编程语言。

# 高阶函数

1. 变量可以指向函数

以Python内置的求绝对值的函数abs()为例，调用该函数用以下代码：

In [1]:
abs(-10)

10

In [2]:
abs

<function abs(x, /)>

In [4]:
fun_abs = abs

函数本身也可以赋值给变量，即：变量可以指向函数。

In [5]:
fun_abs(-10)

10

2. 传入函数

既然变量可以指向函数，函数的参数能接收变量，那么一个函数就可以接收另一个函数作为参数，这种函数就称之为高阶函数。

In [6]:
def add(x, y, f):
    return f(x) + f(y)

In [7]:
add(-5, -6, abs)

11

# map/reduce

我们先看map。map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回。

In [9]:
def f(x):
    return x * x

In [10]:
r = map(f, [1, 2, 3, 4, 5, 6])

In [11]:
r

map()传入的第一个参数是f，即函数对象本身。由于结果r是一个Iterator，Iterator是惰性序列，因此通过list()函数让它把整个序列都计算出来并返回一个list

reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算，其效果就是

reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)

In [12]:
from functools import reduce

In [13]:
def add(x, y):
    return x + y

In [14]:
reduce(add, [1, 2, 3, 4, 5])

15

## filter

In [16]:
def is_odd(n):
    return n % 2 == 1

In [17]:
list(filter(is_odd, [1,2,3,4,5,6,10,15]))

[1, 3, 5, 15]

## sorted

In [19]:
sorted([36, 5, -12, 9, -21])

[-21, -12, 5, 9, 36]

sorted()函数也是一个高阶函数，它还可以接收一个key函数来实现自定义的排序

In [18]:
sorted([36, 5, -12, 9, -21], key=abs)

[5, 9, -12, -21, 36]

## 返回函数

高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。

In [20]:
def calc_sum(*args):
    ax = 0
    for n in args:
        ax = ax + n
    return ax

但是，如果不需要立刻求和，而是在后面的代码中，根据需要再计算怎么办？可以不返回求和的结果，而是返回求和的函数：

In [23]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

当我们调用lazy_sum()时，返回的并不是求和结果，而是求和函数：

In [24]:
f = lazy_sum(1, 3, 5, 7, 9)

In [25]:
f

<function __main__.lazy_sum.<locals>.sum()>

In [26]:
f()

25

在这个例子中，我们在函数lazy_sum中又定义了函数sum，并且，内部函数sum可以引用外部函数lazy_sum的参数和局部变量，当lazy_sum返回函数sum时，相关参数和变量都保存在返回的函数中，这种称为“闭包（Closure)”的程序结构拥有极大的威力。

请再注意一点，当我们调用lazy_sum()时，每次调用都会返回一个新的函数，即使传入相同的参数：

In [27]:
f1 = lazy_sum(1, 3, 5, 7, 9)

In [28]:
f2 = lazy_sum(1, 3, 5, 7, 9)

In [29]:
f1==f2

False

f1()和f2()的调用结果互不影响。

## 闭包

注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。

返回闭包时牢记的一点就是：返回函数不要引用任何循环变量，或者后续会发生变化的变量。

## 匿名函数 lambda

In [31]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

Python对匿名函数的支持有限，只有一些简单的情况下可以使用匿名函数。

## 装饰器

In [32]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

In [35]:
@log
def now():
    print('2015-3-25')

In [36]:
now()

call now():
2015-3-25


如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数

In [37]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

In [38]:
@log('execute')
def now():
    print('2015-3-25')

In [39]:
now()

execute now():
2015-3-25


## 偏函数

在介绍函数参数的时候，我们讲到，通过设定参数的默认值，可以降低函数调用的难度。而偏函数也可以做到这一点。举例如下：

int()函数可以把字符串转换为整数，当仅传入字符串时，int()函数默认按十进制转换：

In [40]:
int('12345')

12345

In [41]:
int('12345', base=8)

5349

In [42]:
int('12345', 16)

74565

假设要转换大量的二进制字符串，每次都传入int(x, base=2)非常麻烦，于是，我们想到，可以定义一个int2()的函数，默认把base=2传进去：

In [43]:
def int2(x, base=2):
    return int(x, base)

In [44]:
int2('1000000')

64

functools.partial就是帮助我们创建一个偏函数的，不需要我们自己定义int2()，可以直接使用下面的代码创建一个新的函数int2

In [45]:
import functools
int2 = functools.partial(int, base=2)

In [46]:
int2('1000000')

64

简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值

In [47]:
int2('1000000', base=10)

1000000

最后，创建偏函数时，实际上可以接收函数对象、*args和**kw这3个参数

In [48]:
int2 = functools.partial(int, base=2)

In [49]:
int2('10010')

18

相当于：

kw = { 'base': 2 }
int('10010', **kw)

当函数的参数个数太多，需要简化时，使用functools.partial可以创建一个新的函数，这个新函数可以固定住原函数的部分参数，从而在调用时更简单。